In [ ]:
# Standard library
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.units import usys
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner, _prev_result_color

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
samples_filename = "../cache/experiment2.h5"
data_filename = "../data/troup-allVisit.h5"
plot_path = "../paper/figures/"
os.makedirs(plot_path, exist_ok=True)

## First read the orbital parameters sampled for this experiment:

In [ ]:
name = os.path.splitext(os.path.split(samples_filename)[1])[0]
hdf5_key = name = '2M00110648+6609349' # HACK: This is hard coded and BAD!!
print(name)

In [ ]:
with h5py.File(samples_filename, 'r') as g:
    jitter = g.attrs['jitter_m/s']*u.m/u.s
    P_min = g.attrs['P_min_day']*u.day
    P_max = g.attrs['P_max_day']*u.day

    # read the orbital parameters
    opars = OrbitalParams.from_hdf5(g)
    samples = opars.pack(plot_units=True)

In [ ]:
print(len(opars))

## read the data:

In [ ]:
with h5py.File(data_filename, 'r') as f:
    g = f[hdf5_key]

    bmjd = g['mjd'][:]
    rv = quantity_from_hdf5(g, 'rv')
    rv_err = quantity_from_hdf5(g, 'rv_err')
data = RVData(bmjd, rv, stddev=rv_err)

### Read in Troup's fit values for the orbital parameters:

In [ ]:
all_troup_data = np.genfromtxt(os.path.join(paths.root, "data", "troup16-dr12.csv"),
                               delimiter=",", names=True, dtype=None)
troup_data = all_troup_data[all_troup_data['APOGEE_ID'].astype(str) == name]

In [ ]:
troup_P = troup_data['PERIOD'] * u.day
troup_ecc = troup_data['ECC']
_,troup_asini = SimulatedRVOrbit.P_K_ecc_to_mf_asini_ecc(troup_P, troup_data['SEMIAMP']*u.m/u.s, troup_ecc)
troup_omega = (troup_data['OMEGA'] % (2*np.pi)) * u.radian

dt0 = atime.Time(troup_data['TPERI'], format='jd', scale='tcb').tcb.mjd - data.t_offset
troup_phi0 = 2*np.pi*dt0/troup_P.value * u.radian

troup_v0 = troup_data['V0'] * u.m/u.s
print(troup_data['SLOPE'])

In [ ]:
troup_truths = np.concatenate((np.log(troup_P.to(u.day).value),
                               np.log(troup_asini.to(u.R_sun).value),
                               troup_ecc,
                               troup_omega.to(u.degree).value % 360.,
                               troup_phi0.to(u.degree).value % 360.,
                               troup_v0.to(u.km/u.s).value))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

dmjd = bmjd.max() - bmjd.min()
t_grid = np.linspace(bmjd.min() -0.25*dmjd, 
                     bmjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

plot_rv_curves(opars[:128], t_grid, rv_unit=rv_unit,
               data=data, ax=ax, 
               plot_kwargs={'color': '#888888', 'zorder': -100},
               data_plot_kwargs={'ecolor': 'k', 'markersize': 3, 'elinewidth': 1, 'alpha': 1., 'zorder': 100})

ax.set_rasterization_zorder(-1)
ax.set_ylim(-40, -18)

ax.set_title(name)
fig.tight_layout()
# fig.savefig(os.path.join(plot_path, 'exp2-rv-curves.pdf'), dpi=100) # change to: dpi=256 for production?

In [ ]:
fig = plt.figure(figsize=(8,6.5))

gs = gridspec.GridSpec(2, 2)

# First, the RV curves
dmjd = bmjd.max() - bmjd.min()
t_grid = np.linspace(bmjd.min() -0.25*dmjd, 
                     bmjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

ax = fig.add_subplot(gs[0, :])
ax.set_title("Experiment 2: {}".format(name))
plot_rv_curves(opars[:128], t_grid, rv_unit=rv_unit,
               ax=ax, t_offset=data.t_offset,
               plot_kwargs={'color': '#888888', 'zorder': -100})

data.plot(ax=ax, rv_unit=rv_unit, ecolor='k', markersize=3, 
          elinewidth=1, alpha=1., zorder=100)
ax.set_ylim(-40, -18)

# Projections of the posterior samples:
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])

style = dict(alpha=0.1, marker=',', linestyle='none', zorder=-100, color='k')
ax2.plot(samples[:,0], samples[:,2], **style)
ax3.plot(samples[:,1], samples[:,2], **style)

# -------------------------------------------------------------
# now plot the truth:
# ax.plot(t_grid, truth_rv.to(rv_unit).value, linestyle='--', 
#         marker=None, linewidth=1, alpha=0.9, color=_truth_color)
ax2.scatter(troup_truths[0], troup_truths[2], marker='+', color=_prev_result_color, s=40, alpha=0.8)
ax3.scatter(troup_truths[1], troup_truths[2], marker='+', color=_prev_result_color, s=40, alpha=0.8)

lnP_lim = [4., 8]
lnasini_lim = [1.5, 5.5]
ecc_lim = [-0.025, 1.025]

ax2.set_xlim(lnP_lim)
ax2.set_ylim(ecc_lim)
ax2.set_xlabel(opars._latex_labels[0])
ax2.set_ylabel(opars._latex_labels[2])

ax3.set_xlim(lnasini_lim)
ax3.set_ylim(ecc_lim)
ax3.set_xlabel(opars._latex_labels[1])

ax2.xaxis.set_ticks(range(4,8+1))
ax3.xaxis.set_ticks(range(2,5+1))
ax3.yaxis.set_ticklabels([])

for ax in fig.axes:
    ax.set_rasterization_zorder(-1)
    
fig.tight_layout()

fig.subplots_adjust(hspace=0.5)

fig.savefig(os.path.join(plot_path, 'exp2-rv-curves.pdf'), dpi=128) # change to: dpi=256 for production?

## Corner

In [ ]:
samples = opars.pack(plot_units=True)

_med_v0 = np.median(samples[:,5])
_mad_v0 = np.median(np.abs(samples[:,5] - _med_v0))

ranges = [
    lnP_lim,
    lnasini_lim,
    ecc_lim,
    (0,360),
    (0,360),
    (_med_v0 - 7*_mad_v0, _med_v0 + 7*_mad_v0)
]

In [ ]:
fig = plot_corner(opars, alpha=0.02, truths=troup_truths, range=ranges, truth_color=_prev_result_color,
                  plot_contours=True, plot_density=True, bins=32)
fig.suptitle("Experiment 2: {}".format(name), fontsize=26)
fig.savefig(os.path.join(plot_path, 'exp2-corner.pdf'), dpi=100) # change to: dpi=256 for production?